### --- Day 17: Chronospatial Computer ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/17

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 17);
WriteLines(inputLines);
// var (inputRegisters, inputInstructions) = inputLines.SeparateBy(line => line is "").ToArray();

// WriteLines(inputRegisters);
// WriteLines(inputInstructions);

Loading puzzle file: Day17.txt
Total lines: 5
Max line length: 40

Register A: 38610541
Register B: 0
Register C: 0

Program: 2,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0


In [4]:
string[] testInputLines =
[
    "Register A: 729",
    "Register B: 0",
    "Register C: 0",
    "",
    "Program: 0,1,5,4,3,0",
];

In [5]:
// delegate int ComboOperand();
delegate void Instruction(int operand);

class Processor
{
    public Processor()
    {
        PopulateInstructions();
    }
    
    // The computer also has three registers named A, B, and C, but these
    // registers aren't limited to 3 bits and can instead hold any integer.
    public int RegisterA { get; private set; }
    public int RegisterB { get; private set; }
    public int RegisterC { get; private set; }

    // A number called the instruction pointer identifies the position in the
    // program from which the next opcode will be read; it starts at 0, pointing at
    // the first 3-bit number in the program.
    public int InstructionPointer { get; private set; }

    public void Process(int a, int b, int c, int[] instructions)
    {
        (RegisterA, RegisterB, RegisterC) = (a, b, c);
        InstructionPointer = 0;
        _output = new();

        while (InstructionPointer < instructions.Length)
        {
            var (inst, operand) = instructions[InstructionPointer..(InstructionPointer+2)];
            var instruction = _instructionLookup[inst];
            instruction(operand);
        }
    }

    StringBuilder _output;
    public string Output => _output.ToString();

    // Operands
    /////

    int Combo(int i) => i switch {
        0 => 0,
        1 => 1,
        2 => 2,
        3 => 3,
        4 => RegisterA,
        5 => RegisterB,
        6 => RegisterC,
        _ => throw new ArgumentException($"Unexpected combo operand")
    };

    // Instructions
    /////

    Dictionary<int, Instruction> _instructionLookup;

    void PopulateInstructions()
    {
        _instructionLookup = new()
        {
            { 0, Adv_0 },
            { 1, Bxl_1 },
            { 2, Bst_2 },
            { 3, Jnz_3 },
            { 4, Bxc_4 },
            { 5, Out_5 },
            { 6, Bdv_6 },
            { 7, Cdv_7 }
        };
    }

    // The eight instructions are as follows:

    // The adv instruction (opcode 0) performs division. The numerator is the
    // value in the A register. The denominator is found by raising 2 to the power
    // of the instruction's combo operand. (So, an operand of 2 would divide A by 4
    // (2^2); an operand of 5 would divide A by 2^B.) The result of the division
    // operation is truncated to an integer and then written to the A register.
    void Adv_0(int operand) {
        RegisterA = RegisterA / (int)Math.Pow(2, Combo(operand));
        InstructionPointer += 2;
    }

    // The bxl instruction (opcode 1) calculates the bitwise XOR of register B
    // and the instruction's literal operand, then stores the result in register B.
    void Bxl_1(int operand) {
        RegisterB = RegisterB ^ operand;
        InstructionPointer += 2;
    }

    // The bst instruction (opcode 2) calculates the value of its combo operand
    // modulo 8 (thereby keeping only its lowest 3 bits), then writes that value to
    // the B register.
    void Bst_2(int operand) {
        RegisterB = Combo(operand) % 8;
        InstructionPointer += 2;
    }

    // The jnz instruction (opcode 3) does nothing if the A register is 0.
    // However, if the A register is not zero, it jumps by setting the instruction
    // pointer to the value of its literal operand; if this instruction jumps, the
    // instruction pointer is not increased by 2 after this instruction.
    void Jnz_3(int operand) {
        if (RegisterA is 0) {
            InstructionPointer += 2;
            return; 
        }
        InstructionPointer = operand;
    }

    // The bxc instruction (opcode 4) calculates the bitwise XOR of register B
    // and register C, then stores the result in register B. (For legacy reasons,
    // this instruction reads an operand but ignores it.)
    void Bxc_4(int operand) {
        RegisterB = RegisterB ^ RegisterC;
        InstructionPointer += 2;
    }

    // The out instruction (opcode 5) calculates the value of its combo operand
    // modulo 8, then outputs that value. (If a program outputs multiple values,
    // they are separated by commas.)
    void Out_5(int operand) {
        if (_output.Length > 0) {
            _output.Append(",");
        }
        _output.Append(Combo(operand) % 8);
        InstructionPointer += 2;
    }

    // The bdv instruction (opcode 6) works exactly like the adv instruction
    // except that the result is stored in the B register. (The numerator is still
    // read from the A register.)
    void Bdv_6(int operand) {
        RegisterB = RegisterA / (int)Math.Pow(2, Combo(operand));
        InstructionPointer += 2;
    }

    // The cdv instruction (opcode 7) works exactly like the adv instruction
    // except that the result is stored in the C register. (The numerator is still
    // read from the A register.)
    void Cdv_7(int operand) {
        RegisterC = RegisterA / (int)Math.Pow(2, Combo(operand));
        InstructionPointer += 2;
    }
}

In [6]:
string RunProcessor(string[] inputLines)
{
    var numbers = inputLines.ParseAll(@"\d+").Select(int.Parse).ToArray();

    var (a, b, c) = numbers;
    var instructions = numbers[3..].ToArray();
    
    Processor processor = new();
    processor.Process(a, b, c, instructions);
    return processor.Output;
}

In [7]:
// After the above program halts, its final output will be 4,6,3,5,6,3,5,2,1,0.

var testAnswer = RunProcessor(testInputLines);
Console.WriteLine(testAnswer);

4,6,3,5,6,3,5,2,1,0


In [8]:
var part1Answer = RunProcessor(inputLines);
Console.WriteLine(part1Answer);

7,5,4,3,4,5,3,4,6


In [9]:
// 7,5,4,3,4,5,3,4,6 is correct
Ensure("7,5,4,3,4,5,3,4,6", part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/17